In [2547]:
import numpy as np
from PIL import Image, ImageDraw, ImageColor

In [2548]:
class RotationTranslationMatrix:
    def __init__(self, m) -> None:
        assert m.shape[0] == m.shape[1]
        self.m = m

In [2549]:
class Vector:
    def __init__(self, v):
        self.v = v

    def __eq__(self, __o):
        return np.allclose(self.v, __o.v)

    def __add__(self, __o):
        return Vector(self.v + __o.v)

    def __sub__(self, __o):
        return Vector(self.v - __o.v)

    def __mul__(self, __o):
        return Vector(self.v * __o)
    
    def to_point(self):
        return Point(nparr=self.v)

    def norm(self):
        return np.linalg.norm(self.v)

    def unit(self):
        return self.v / self.norm()

    def __repr__(self) -> str:
        return f"Vector {self.v}"

In [2550]:
class Point:
    def __init__(self, *xyz, nparr=None):
        self.xyz = np.array(xyz) if xyz else nparr

    def as_affine(self):
        self.xyz = self.xyz[:-1] / self.xyz[-1]

    def as_homogenious(self):
        self.xyz = np.append(self.xyz, 1)  # return new?

    def to_vec(self):
        return Vector(self.xyz)

    def __eq__(self, __o):
        return np.allclose(self.xyz, __o.xyz)

    def __add__(self, __o: Vector):
        return Point(nparr=self.xyz + __o.v)

    def __sub__(self, __o):
        return Vector(self.xyz - __o.xyz)

    # def __mul__(self, __o):
    #     return Point(self.xyz * __o)

    # def __matmul__(self, m):
    #     return Vector(self.as_homogenious() @ m).to_affine()

    def __repr__(self) -> str:
        return f"Point {self.xyz}"

In [2551]:
class Edge:
    def __init__(
        self,
        a,
        b,
    ):
        self.a = a
        self.b = b
        self.v = b - a

    def __eq__(self, __o):
        return (
            self.a == __o.a and self.b == __o.b or self.a == __o.b and self.b == __o.a
        )

    def normal(self, viewpoint):
        v_a = self.a - viewpoint
        v_b = self.b - self.a
        proj = v_b * np.dot(v_a.v, v_b.v) * (1 / v_b.norm() ** 2)
        n = (v_a - proj).unit()
        return n

    def __repr__(self) -> str:
        return f"Edge <{self.a} {self.b}>"

In [2552]:
# TODO
# 2) DEFINE NORMALS FOR EDGES
# 1) DELAUNEY TRIANGUALTION

import functools
import operator


class Polygon:
    def __init__(self, **kwargs) -> None:

        self.vertices = None
        self.edges = None
        self.rotation = None
        self.L = None
        self.convex_type = None
        self.simple_type = None
        self.signed_area = None
        self.triangluation = None
        self.mean_center = None
        self.edge_normals = None
        self.simple_polygon_list = []
        self.color = "black"
        self.__dict__.update(kwargs)

        assert self.vertices or self.edges, "Polygon: vertices and edges are empty"

        if self.edges is None:
            self._edges()

        if self.vertices is None:
            self._vertices()

        if self.L is None:
            self.L = len(self.vertices)

        if self.convex_type is None:
            self._convex_type()

        if self.simple_type is None:
            self._simple_type()

        if self.simple_polygon_list == []:
            if self.simple_type == "SELF_INTERSECTED":
                self._simple_polygon_list()
            else:
                self.simple_polygon_list = [self]

        if self.triangluation is None:
            self._triangluation()
        
        if self.signed_area is None:
            self._signed_area()

        if self.rotation is None:
            self._rotation()
        
        if self.mean_center is None:
            self._mean_center()

        if self.edge_normals is None:
            self._edge_normals()

        self.color_rgb = ImageColor.getrgb(self.color)

    def _edges(self):
        self.edges = [
            Edge(p1, p2)
            for p1, p2 in zip(self.vertices, self.vertices[1:] + [self.vertices[0]])
        ]

    def _vertices(self):
        self.vertices = [edge.a for edge in self.edges]

    def _convex_type(self):
        flag = None
        for i in range(self.L):
            for j in range(i + 1, i + self.L - 1):
                cl_type = classifyEdgePoint(self.edges[j % self.L], self.vertices[i])
                if flag is None:
                    flag = cl_type
                elif flag != cl_type:
                    self.convex_type = "CONCAVE"
                    return
        self.convex_type = "CONVEX"

    def _simple_type(self):
        flag = "SIMPLE"
        intersection_pairs = {}
        for i in range(self.L - 1):
            for j in range(i + 2, self.L):
                if i == (j + 1) % self.L:
                    continue
                vi, vj = self.edges[i], self.edges[j]
                cross_type, ti = crossEdges(vi, vj)
                if cross_type == "SKEW_CROSS":
                    intersection_point = vi.a + (vi.b - vi.a) * ti
                    tj = (intersection_point - vj.a).norm() / vj.v.norm()
                    intersection_pairs[(i, j)] = (intersection_point, ti, tj)
                    flag = "SELF_INTERSECTED"
        self.simple_type = flag
        self._intersection_pairs = intersection_pairs

    def _rotation(self):
        self.rotation = "CCW" if self.signed_area > 0 else "CW"

    def _simple_polygon_list(self):
        # Step 1: Collect intersection points per edge
        splits = {}  # edge_idx -> [(t_param, point), ...]
        for (i, j), (point, ti, tj) in self._intersection_pairs.items():
            for idx, t in ((i, ti), (j, tj)):
                splits.setdefault(idx, []).append((t, point))

        # Sort splits by parameter t for each edge
        for split_list in splits.values():
            split_list.sort(key=lambda x: x[0])

        # Build new edges from sorted splits
        split_edges = {}
        for idx, split_list in splits.items():
            edge = self.edges[idx]
            segments = [edge.a] + [pt for _, pt in split_list] + [edge.b]
            split_edges[idx] = [
                Edge(segments[i], segments[i + 1]) for i in range(len(segments) - 1)
            ]

        stack_edges = [[]]
        for i in range(self.L):
            if i not in split_edges:
                stack_edges[-1].append(self.edges[i])
                if (
                    len(stack_edges[-1]) > 1 and \
                        stack_edges[-1][0].a == stack_edges[-1][-1].b
                ):
                    self.simple_polygon_list.append(
                        Polygon(
                            edges=stack_edges.pop(), 
                            simple_type="SIMPLE",
                            color=self.color)
                        )
            else:
                for edge in split_edges[i]:
                    stack_edges[-1].append(edge)
                    if (
                        len(stack_edges[-1]) > 1 and \
                            stack_edges[-1][0].a == stack_edges[-1][-1].b
                    ):
                        self.simple_polygon_list.append(
                            Polygon(
                                edges=stack_edges.pop(), 
                                simple_type="SIMPLE",
                                color=self.color
                                )
                            )
                    else:
                        stack_edges.append([])
                if stack_edges:
                    del stack_edges[-1]

    def _signed_area(self):
        self.signed_area = 0.5 * sum(np.cross(edge.a.xyz, edge.b.xyz) for edge in self.edges)
    
    def _edge_normals(self):
        self.edge_normals = [edge.normal(viewpoint=self.mean_center) for edge in self.edges]

    def EO(self, p):
        intersection_sum = 0
        for edge in self.edges:
            match classifyEdgePoint(edge, p):
                case "TOUCHING":
                    return "INSIDE"
                case "CROSS_LEFT":
                    intersection_sum += 1
                case "CROSS_RIGHT":
                    intersection_sum += 1
        if intersection_sum % 2 == 1:
            return "INSIDE"
        return "OUTSIDE"

    def NZW(self, p):
        winding_number = 0
        for edge in self.edges:
            match classifyEdgePoint(edge, p):
                case "TOUCHING":
                    return "INSIDE"
                case "CROSS_LEFT":
                    winding_number += 1
                case "CROSS_RIGHT":
                    winding_number -= 1
        if winding_number == 0:
            return "OUTSIDE"
        return "INSIDE"
    
    def _mean_center(self):
        vecs = (v.to_vec() for v in self.vertices)
        self.mean_center = (functools.reduce(operator.add, vecs) * (1 / self.L)).to_point()
    
    def _visible_edges(self, view_direction):
        ...

    def _triangluation(self):
        # for pairs in zip(self.vertices, self.edges):
        #     # Отсортируем все точки вдоль некоторой прямой (для простоты по координате $x$).
        #     pairs = sorted(pairs, key= lambda x: x[0].xyz[0])
        #     # Построим треугольник на первых 3 точках.
        #     triangle = Polygon(edges=pairs[:3])
        ...
            

In [2553]:
def edge_visible(): 
    ...

In [2554]:
def check_edge_visibility(face, view_direction=np.array([0, 0, 1000])):
    v1, v2, v3, *_ = face
    normal = np.cross(np.array([*(v3 - v1)]), np.array([*(v2 - v1)]))
    dot_product = np.dot(normal, view_direction)
    return dot_product <= 0

In [2555]:
def draw_polygon(polygon, draw):
    for edge in polygon.edges:
        draw.point(pixelEdge(edge), fill=polygon.color)

In [2556]:
def pixelLine(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    ix = 1 if dx > 0 else -1
    iy = 1 if dy > 0 else -1
    dx *= ix
    dy *= iy
    x = x1
    y = y1
    pixels = []
    if dx < dy:
        E = 2 * dx - dy
        for _ in range(dy + 1):
            pixels.append((x, y))
            if E >= 0:
                x += ix
                E -= 2 * dy
            y += iy
            E += 2 * dx
    else:
        E = 2 * dy - dx
        for _ in range(dx + 1):
            pixels.append((x, y))
            if E >= 0:
                y += iy
                E -= 2 * dx
            x += ix
            E += 2 * dy
    return pixels


def pixelEdge(e):
    x1, y1, x2, y2 = (
        round(e.a.xyz[0]),
        round(e.a.xyz[1]),
        round(e.b.xyz[0]),
        round(e.b.xyz[1]),
    )
    return pixelLine(x1, y1, x2, y2)

In [2557]:
def classify(x1, y1, x2, y2, x, y):
    ax = x2 - x1
    ay = y2 - y1
    bx = x - x1
    by = y - y1
    s = ax * by - ay * bx
    if s > 0:
        return "LEFT"
    if s < 0:
        return "RIGHT"
    if ax * bx < 0 or ay * by < 0:
        return "BEHIND"
    if ax**2 + ay**2 < bx**2 + by**2:
        return "INFRONT"
    if x == x1 and y == y1:
        return "ORIGIN"
    if x == x2 and y == y2:
        return "DESTINATION"
    return "BETWEEN"


def classifyEdgePoint(e, p):
    return classify(*e.a.xyz, *e.b.xyz, *p.xyz)

In [2558]:
def intersect(ax, ay, bx, by, cx, cy, dx, dy):
    nx = dy - cy
    ny = cx - dx
    denom = nx * (bx - ax) + ny * (by - ay)
    if denom == 0:
        type = classify(cx, cy, dx, dy, ax, ay)
        if type == "LEFT" or type == "RIGHT":
            return "PARALLEL", None
        else:
            return "SAME", None
    num = nx * (ax - cx) + ny * (ay - cy)
    t = -num / denom
    return "SKEW", t


def intersectEdges(e1, e2):
    return intersect(*e1.a.xyz, *e1.b.xyz, *e2.a.xyz, *e2.b.xyz)

In [2559]:
def cross(ax, ay, bx, by, cx, cy, dx, dy):
    type, tab = intersect(ax, ay, bx, by, cx, cy, dx, dy)
    if type == "SAME" or type == "PARALLEL" or tab is None:
        return type, None
    if tab < 0 or tab > 1:
        return "SKEW_NO_CROSS", None
    _, tcd = intersect(cx, cy, dx, dy, ax, ay, bx, by)
    if tcd is not None and (tcd < 0 or tcd > 1):
        return "SKEW_NO_CROSS", None
    return "SKEW_CROSS", tab


def crossEdges(e1, e2):
    return cross(*e1.a.xyz, *e1.b.xyz, *e2.a.xyz, *e2.b.xyz)

In [2560]:
img = Image.new("RGB", (200, 200), "white")
draw = ImageDraw.Draw(img)

In [2561]:
p0 = Polygon(
    vertices=[
        Point(150, 80),
        Point(10, 60),
        Point(50, 100),
        Point(100, 20),
        Point(120, 160),
        Point(190, 100),
        Point(180, 190),
    ],
    color="blue",
)

In [2562]:
p0.convex_type

'CONCAVE'

In [2563]:
p0.simple_type

'SELF_INTERSECTED'

In [2564]:
p0.simple_polygon_list

In [2565]:
p0.signed_area

-1800.0

In [2566]:
p0.rotation

'CW'

In [2567]:
draw_polygon(p0, draw)

In [2568]:
p1 = Polygon(
    vertices=[
        Point(10, 10),
        Point(40, 100),
        Point(100, 150),
        Point(50, 180),
        Point(70, 10),
    ],
    color="green",
)

In [2569]:
p1.convex_type

'CONCAVE'

In [2570]:
p1.simple_type

'SELF_INTERSECTED'

In [2571]:
p1.simple_polygon_list

[<__main__.Polygon at 0x160b5a1e090>, <__main__.Polygon at 0x160b61764b0>]

In [2572]:
# draw_polygon(p1, draw)

In [2573]:
p2 = Polygon(
    vertices=[
        Point(20, 10),
        Point(50, 10),
        Point(80, 20),
        Point(90, 50),
        Point(70, 80),
        Point(40, 90),
        Point(10, 60),
    ],
    color="yellow",
)

In [2574]:
p2.convex_type

'CONVEX'

In [2575]:
p2.simple_type

'SIMPLE'

In [2576]:
p2.simple_polygon_list

In [2577]:
# draw_polygon(p2, draw)
# img.show()

In [2578]:
triangle = Polygon(
    vertices=[
        Point(34, 80),
        Point(57, 1),
        Point(170, 80),
    ],
    color="yellow",  # A B  # B C  # C A
)

In [2579]:
def paint_triangle():
    edges = [pixelEdge(edge) for edge in triangle.edges]
    ABC = edges[0] + edges[1]
    i = 0
    AC = edges[2][::-1]
    j = 0
    A = AC[0]
    C = AC[-1]
    dxAC = abs(A[0] - C[0])
    dyAC = abs(A[1] - C[1])
    k = 0 if dxAC > dyAC else 1
    pixels = []
    while ABC[i] != C:
        line = pixelLine(*ABC[i], *AC[j])
        pixels.extend(line)
        i += 1
        if AC[j][k] != ABC[i][k]:
            j += 1
    return pixels

In [2580]:
# draw.point(paint_triangle(), fill="red")


img.show()